In [1]:
from math import sqrt

import numpy as np

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import skimage
from skimage import io, exposure, img_as_float, img_as_ubyte, morphology, filters, util
from skimage.color import rgb2gray, label2rgb
from skimage.feature import canny, blob_dog, blob_log, blob_doh,  peak_local_max
from skimage.filters import sobel, threshold_otsu, try_all_threshold, threshold_local, threshold_minimum
from skimage.segmentation import clear_border, watershed, random_walker
from skimage.measure import label, regionprops
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.morphology import closing

import cv2

from scipy import ndimage as ndi
from scipy.stats import norm

import os

import plotly.graph_objs as go

import pickle

In [7]:
!pip install numpy==1.16

  Using cached https://files.pythonhosted.org/packages/e4/50/380aebcda065f62febb99fd5a7253d27d9f10719c5d90938ee642b4fee54/numpy-1.16.0-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
distributed 1.21.8 requires msgpack, which is not installed.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
cryptography 3.3.1 has requirement cffi>=1.12, but you'll have cffi 1.11.5 which is incompatible.
  Found existing installation: numpy 1.15.0
    Uninstalling numpy-1.15.0:
      Successfully uninstalled numpy-1.15.0
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
with open("/Users/carolinemccue/Desktop/CellArea/r01c01f01p01-ch2sk1fk1fl1/r01c01f01p01-ch2sk1fk1fl1.pkl", 'rb') as f:
    area2p5 = pickle.load(f)

area2p5

TypeError: scalar() argument 1 must be numpy.dtype, not numpy.dtype

In [3]:
# 2.5 um

with open("/Users/carolinemccue/Desktop/MG63/r01c01f01p01-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts2p5 = pickle.load(f)
    
posts2p5 = {k:v for k,v in num_posts2p5.items() if v != 'Delete'}


In [ ]:
# 5 um

with open("/Users/carolinemccue/Desktop/MG63/r01c02f26p03-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts5 = pickle.load(f)
    
posts5 = {k:v for k,v in num_posts5.items() if v != 'Delete'}

posts5

In [ ]:
# 10 um

with open("/Users/carolinemccue/Desktop/MG63/r01c03f60p02-ch2sk1fk1fl1/labeled_carolinemccue.pkl", 'rb') as f:
    num_posts10 = pickle.load(f)
    
posts10 = {k:v for k,v in num_posts10.items() if v != 'Delete'}